# TFM

data: https://drive.google.com/drive/folders/1fx_j6gmiATvhEaBtUwEPuCOl8peJ2zDM

## 1. Simple k-NN implementation

In [108]:
import math
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import time
import statistics as st
from scipy.spatial import distance as dt
import distance
from operator import truediv, add, mul

### Euclidean distance

In [50]:
# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return math.sqrt(distance)

#### Time comparison

In [51]:
row0 = df_tstrss_DSI1[0]
start = time.time()
for row in df_trnrss_DSI1:
    euclidean_distance(row0, row)
end = time.time()
print(end - start)

0.0727691650390625


In [ ]:
a = np.array(df_tstrss_DSI1[0])
b = np.array(df_trnrss_DSI1)
start = time.time()
for row in b:
    np.linalg.norm(a-b)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
for row in df_trnrss_DSI1:
    np.sqrt(np.sum(np.square(a-b)))
end = time.time()
print(end - start)

In [ ]:
from scipy.spatial import distance
start = time.time()
for row in df_trnrss_DSI1:
    distance.euclidean(row0, row)
end = time.time()
print(end - start)

In [53]:
# WINNER
start = time.time()
for row in df_trnrss_DSI1:
    math.dist(row0, row)
end = time.time()
print(end - start)

0.013959169387817383


### Get k-nearest neighbors

In [ ]:
# aquí hi va la funció get_neighbors()

#### Time comparison

In [ ]:
start = time.time()
neighbors = get_neighbors(df_trnrss_DSI1, row0, 3)
end = time.time()
print(end - start)

### Data from csv

In [4]:
def load_Data(name):
    # TRAIN data
    # load fingerprints
    df_trnrss_DSI1 = pd.read_csv('data/'+ name + '_trnrss.csv', header=None).values.tolist()
    # load coordinates
    df_trncrd_DSI1 = pd.read_csv('data/' + name + '_trncrd.csv', header=None)

    # for now, drop columns of 'floor' and 'building', keep x,y,z
    df_trncrd_DSI1.drop(df_trncrd_DSI1.columns[[3,4]], axis=1, inplace=True)
    df_trncrd_DSI1 = df_trncrd_DSI1.values.tolist()

    # TEST data
    # load fingerprints
    df_tstrss_DSI1 = pd.read_csv('data/' + name + '_tstrss.csv', header=None).values.tolist()
    # load coordinates
    df_tstcrd_DSI1 = pd.read_csv('data/' + name + '_tstcrd.csv', header=None)

    # for now, drop columns of 'floor' and 'building', keep x,y,z
    df_tstcrd_DSI1.drop(df_tstcrd_DSI1.columns[[3,4]], axis=1, inplace=True)
    df_tstcrd_DSI1 = df_tstcrd_DSI1.values.tolist()
    
    return(df_trnrss_DSI1, df_trncrd_DSI1, df_tstrss_DSI1, df_tstcrd_DSI1)

In [5]:
df_trnrss_DSI1, df_trncrd_DSI1, df_tstrss_DSI1, df_tstcrd_DSI1 = load_Data("DSI1")
df_trnrss_DSI2, df_trncrd_DSI2, df_tstrss_DSI2, df_tstcrd_DSI2 = load_Data("DSI2")
df_trnrss_LIB1, df_trncrd_LIB1, df_tstrss_LIB1, df_tstcrd_LIB1 = load_Data("LIB1")
df_trnrss_LIB2, df_trncrd_LIB2, df_tstrss_LIB2, df_tstcrd_LIB2 = load_Data("LIB2")
df_trnrss_MAN2, df_trncrd_MAN2, df_tstrss_MAN2, df_tstcrd_MAN2 = load_Data("MAN2")
df_trnrss_SIM001, df_trncrd_SIM001, df_tstrss_SIM001, df_tstcrd_SIM001 = load_Data("SIM001")

### Algorithm

In [6]:
# calculate knn of fingerprints
def knn(k, tstrss, trnrss):
    all_neighbors = []

    # iterate test data and find id of k nearest neighbors from trnrss
    for i in tstrss:
        all_neighbors.append(get_neighbors(trnrss, i, k))
        
    return all_neighbors

# predict position of neighbors 
def predict_position(all_neighbors, trncrd):
    predicted_pos = []
    
    # iterate all neighbors to predict position. Find k indexes in trncrd and calculate mean value.
    for knn in all_neighbors:    
        x, y, z = 0, 0, 0
        for i in knn:
            x += trncrd[i][0]
            y += trncrd[i][1]
            z += trncrd[i][2]
        predicted_pos.append([x/k, y/k, round(z/k, 1)]) # mean of the error
        
    return predicted_pos

# error calculation: returns euclidean distance between predictions and tstcrd
def calculate_error(predicted_pos, tstcrd):
    error_distances = [math.dist(predicted_pos[i], tstcrd[i]) for i, val in enumerate(predicted_pos)]
    return error_distances

# calculates knn and returns prediction of position
def knn_and_prediction(k, tstrss, trnrss, trncrd):
    neighbors = knn(k, tstrss, trnrss)
    return predict_position(neighbors, trncrd)

# calculates knn, predicts position and returns error distances between prediction and real position
def positioning_error(k, tstrss, trnrss, tstcrd, trncrd):
    positions = knn_and_prediction(k, tstrss, trnrss, trncrd)
    return calculate_error(positions, tstcrd)

### Calculations

In [ ]:
start = time.time()
k=1
k1_DSI1 = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
k1_DSI2 = positioning_error(k, df_tstrss_DSI2, df_trnrss_DSI2, df_tstcrd_DSI2, df_trncrd_DSI2)
k1_LIB1 = positioning_error(k, df_tstrss_LIB1, df_trnrss_LIB1, df_tstcrd_LIB1, df_trncrd_LIB1)
k1_LIB2 = positioning_error(k, df_tstrss_LIB2, df_trnrss_LIB2, df_tstcrd_LIB2, df_trncrd_LIB2)
k1_MAN2 = positioning_error(k, df_tstrss_MAN2, df_trnrss_MAN2, df_tstcrd_MAN2, df_trncrd_MAN2)
k1_SIM001 = positioning_error(k, df_tstrss_SIM001, df_trnrss_SIM001, df_tstcrd_SIM001, df_trncrd_SIM001)

k=5
k5_DSI1 = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
k5_DSI2 = positioning_error(k, df_tstrss_DSI2, df_trnrss_DSI2, df_tstcrd_DSI2, df_trncrd_DSI2)
k5_LIB1 = positioning_error(k, df_tstrss_LIB1, df_trnrss_LIB1, df_tstcrd_LIB1, df_trncrd_LIB1)
k5_LIB2 = positioning_error(k, df_tstrss_LIB2, df_trnrss_LIB2, df_tstcrd_LIB2, df_trncrd_LIB2)
k5_MAN2 = positioning_error(k, df_tstrss_MAN2, df_trnrss_MAN2, df_tstcrd_MAN2, df_trncrd_MAN2)
k5_SIM001 = positioning_error(k, df_tstrss_SIM001, df_trnrss_SIM001, df_tstcrd_SIM001, df_trncrd_SIM001)

k=11
k11_DSI1 = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
k11_DSI2 = positioning_error(k, df_tstrss_DSI2, df_trnrss_DSI2, df_tstcrd_DSI2, df_trncrd_DSI2)
k11_LIB1 = positioning_error(k, df_tstrss_LIB1, df_trnrss_LIB1, df_tstcrd_LIB1, df_trncrd_LIB1)
k11_LIB2 = positioning_error(k, df_tstrss_LIB2, df_trnrss_LIB2, df_tstcrd_LIB2, df_trncrd_LIB2)
k11_MAN2 = positioning_error(k, df_tstrss_MAN2, df_trnrss_MAN2, df_tstcrd_MAN2, df_trncrd_MAN2)
k11_SIM001 = positioning_error(k, df_tstrss_SIM001, df_trnrss_SIM001, df_tstcrd_SIM001, df_trncrd_SIM001)
end = time.time()
print(end - start)

In [14]:
start = time.time()
k=1
k1_DSI1 = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
k=5
k5_DSI1 = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
k=11
k11_DSI1 = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
end = time.time()
print(end - start)

10.582464218139648


### ECDF

In [7]:
#ECDF function to generate x and y axis data
def ecdf(xdata):
    xdataecdf = np.sort(xdata)
    ydataecdf = np.arange(1, len(xdata) + 1) / len(xdata)
    return xdataecdf, ydataecdf

def plot_ecdf(data1, data2, data3, title, figure):
    #Get the x and y data for ecdf plot from ecdf method
    x1,y1 = ecdf(data1)
    x5,y5 = ecdf(data2)
    x11,y11 = ecdf(data3)

    #Plot the data using matplotlib
    plt.figure(figure)
    plt.plot(x1, y1, marker = '.', linestyle = 'none', markersize = 3)
    plt.plot(x5, y5, marker = '.', linestyle = 'none', markersize = 3)
    plt.plot(x11, y11, marker = '.', linestyle = 'none', markersize = 3)
    plt.legend(("k=1", "k=5", "k=11"))
    plt.title(title)
    plt.xlabel('Error distance')
    plt.ylabel('Error probability')
    plt.margins(0.1)

In [ ]:
plot_ecdf(k1_DSI1, k5_DSI1, k11_DSI1, "DSI1", 1)
plot_ecdf(k1_DSI2, k5_DSI2, k11_DSI2, "DSI2", 2)
plot_ecdf(k1_LIB1, k5_LIB1, k11_LIB1, "LIB1", 3)
plot_ecdf(k1_LIB2, k5_LIB2, k11_LIB2, "LIB2", 4)
plot_ecdf(k1_MAN2, k5_MAN2, k11_MAN2, "MAN2", 5)
plot_ecdf(k1_SIM001, k5_SIM001, k11_SIM001, "SIM001", 6)

In [ ]:
#Import seaborn library and generate ecdf plot here
sns.ecdfplot(x = k1_LIB1).set_title('LIB1');
sns.ecdfplot(x = k5_LIB1);
sns.ecdfplot(x = k11_LIB1);

### Execution time for knn and prediction

In [15]:
k = 1

# calculate knn and predict positions
start = time.time()
positions_DSI1 = knn_and_prediction(k, df_tstrss_DSI1, df_trnrss_DSI1, df_trncrd_DSI1)
end = time.time()
print("DSI1: {}".format(end - start))

start = time.time()
positions_DSI2 = knn_and_prediction(k, df_tstrss_DSI2, df_trnrss_DSI2, df_trncrd_DSI2)
end = time.time()
print("DSI2: {}".format(end - start))

start = time.time()
positions_LIB1 = knn_and_prediction(k, df_tstrss_LIB1, df_trnrss_LIB1, df_trncrd_LIB1)
end = time.time()
print("LIB1: {}".format(end - start))

start = time.time()
positions_LIB2 = knn_and_prediction(k, df_tstrss_LIB2, df_trnrss_LIB2, df_trncrd_LIB2)
end = time.time()
print("LIB2: {}".format(end - start))

start = time.time()
positions_MAN2 = knn_and_prediction(k, df_tstrss_MAN2, df_trnrss_MAN2, df_trncrd_MAN2)
end = time.time()
print("MAN2: {}".format(end - start))

start = time.time()
positions_SIM001 = knn_and_prediction(k, df_tstrss_SIM001, df_trnrss_SIM001, df_trncrd_SIM001)
end = time.time()
print("SIM001: {}".format(end - start))

DSI1: 3.59747052192688
DSI2: 1.508279800415039
LIB1: 7.516261577606201
LIB2: 8.368971347808838
MAN2: 0.4539635181427002
SIM001: 7.518806457519531


### Calculate average error

In [18]:
# calculate average error from list of positioning errors
averr_DSI1 = st.fmean(k1_DSI1)
averr_DSI2 = st.fmean(k1_DSI2)
averr_LIB1 = st.fmean(k1_LIB1)
averr_LIB2 = st.fmean(k1_LIB2)
averr_MAN2 = st.fmean(k1_MAN2)
averr_SIM001 = st.fmean(k1_SIM001)

In [19]:
print("DSI1: {}".format(round(averr_DSI1, 2)))
print("DSI2: {}".format(round(averr_DSI2, 2)))
print("LIB1: {}".format(round(averr_LIB1, 2)))
print("LIB2: {}".format(round(averr_LIB2, 2)))
print("MAN2: {}".format(round(averr_MAN2, 2)))
print("SIM001: {}".format(round(averr_SIM001, 2)))

DSI1: 6.46
DSI2: 6.46
LIB1: 3.8
LIB2: 4.47
MAN2: 3.26
SIM001: 3.37


### Distance measures comparison on DSI1: average error

In [58]:
df_tstrss_DSI1[0]-df_tstrss_DSI1[23]

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [131]:
def normalize(h):
    return h / np.sum(h)

def gower_dist(P, Q):
    P = np.array(P)
    Q = np.array(Q)
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    return sum(abs_subs)/len(P)

def soergel_dist(P, Q):
    P = np.array(P)
    Q = np.array(Q)
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    max_el = list(map(max, P, Q))
    return sum(abs_subs)/sum(max_el)

def kulczynski_d_dist(P, Q):
    P = np.array(P)
    Q = np.array(Q)
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    min_el = list(map(min, P, Q))
    return sum(abs_subs)/sum(min_el)

def lorentzian_dist(P, Q):
    P = np.array(P)
    Q = np.array(Q)
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    add_1 = list(np.asarray(abs_subs) + 1)
    return sum(np.log(add_1))

def intersection_dist(P, Q):
    return sum(list(map(min, P, Q)))

def wavehedges(P, Q):
    min_el = list(map(min, P, Q))
    subs = list(1 - np.asarray(min_el))
    max_el = list(map(max, P, Q))
    return sum(list(map(truediv, subs, max_el)))

def czekanowski_s_dist(P, Q):
    min_el = list(map(min, P, Q))
    subs = list(1 - np.asarray(min_el))
    sumPQ = list(map(add, P, Q) )
    return 2*sum(list(map(truediv, subs, sumPQ)))

def czekanowski_d_dist(P, Q):
    return 1 - czekanowski_s_dist(P, Q)

def motyka_s_dist(P, Q):
    min_el = list(map(min, P, Q))
    subs = list(1 - np.asarray(min_el))
    sumPQ = list(map(add, P, Q))
    return sum(list(map(truediv, subs, sumPQ)))

def motyka_d_dist(P, Q):
    return 1 - motyka_s_dist(P, Q)

def kulczynski_s_dist(P, Q):
    P = np.array(P)
    Q = np.array(Q)
    abs_subs = list(map(abs, list(np.subtract(P, Q))))
    min_el = list(map(min, P, Q))
    dist = 0
    
    if sum(abs_subs) != 0:
        dist = sum(list(map(truediv, min_el, abs_subs)))
    
    if sum(min_el) != 0:
        dist = 1/dist
    
    return dist

def ruzicka_dist(P, Q):
    min_el = list(map(min, P, Q))
    max_el = list(map(max, P, Q))
    return 1 - sum(min_el)/sum(max_el)
        
def tanimoto_dist(P, Q):
    min_el = list(map(min, P, Q))
    return (sum(P) + sum(Q) - 2*sum(min_el)) /  (sum(P) + sum(Q) - sum(min_el))

def harmonic(P, Q):
    min_el = list(map(min, P, Q))
    multPQ = list(map(mul, P, Q))
    sumPQ = list(map(add, P, Q))
    return 2*sum(list(map(truediv, multPQ, sumPQ)))

def kumar(P, Q):
    return np.dot(P, Q) / (np.dot(P, P) + np.dot(Q, Q) - np.dot(P, Q))

def jaccard_s_dist(P, Q):
    return kumar(P, Q)
        
def jaccard_d_dist(P, Q):
    return 1 - jaccard_s_dist(P, Q)

def dice_s_dist(P, Q):
    return 2*np.dot(P, Q) / (np.dot(P, P) + np.dot(Q, Q))
    
def dice_d_dist(P, Q):
    return 1 - dice_s_dist(P, Q)
    
# Locate the most similar neighbors and return list of indexes
def get_neighbors(train, test_row, k):
    distances = list()
    for idx, train_row in enumerate(train):
        # Lp Minkowski family
        #dist = math.dist(test_row, train_row) #euclidean
        #dist = dt.minkowski(test_row, train_row, 5)
        #dist = dt.cityblock(test_row, train_row)
        #dist = dt.chebyshev(test_row, train_row)
        
        # L1 family
        #dist = distance.sorensen(test_row, train_row)
        #dist = gower_dist(test_row, train_row)
        #dist = soergel_dist(test_row, train_row)
        #dist = kulczynski_d_dist(test_row, train_row)
        #dist = lorentzian_dist(test_row, train_row)
        #dist = dt.canberra(test_row, train_row)
        
        # Intersection family
        #dist = intersection_dist(test_row, train_row)
        #dist = wavehedges(test_row, train_row)
        #dist = czekanowski_s_dist(test_row, train_row)
        #dist = czekanowski_d_dist(test_row, train_row)
        #dist = motyka_s_dist(test_row, train_row)
        #dist = motyka_d_dist(test_row, train_row)
        #dist = kulczynski_s_dist(test_row, train_row)
        #dist = ruzicka_dist(test_row, train_row)
        #dist = tanimoto_dist(test_row, train_row)
        
        # Inner product family
        #dist = np.inner(test_row, train_row)
        #dist = harmonic(test_row, train_row)
        #dist = dt.cosine(test_row, train_row)
        #dist = kumar(test_row, train_row)
        #dist = jaccard_s_dist(test_row, train_row)
        #dist = jaccard_d_dist(test_row, train_row)
        #dist = dice_s_dist(test_row, train_row)
        dist = dice_d_dist(test_row, train_row)
        
        # Fidelity family or Squared-chord family
        #dist = 1 - np.sum(np.sqrt(np.multiply(normalize(test_row), normalize(train_row))))
        
        distances.append((idx, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = [distances[i][0] for i in range(k)]
    return neighbors

#### Lp Minkowski family

In [10]:
# Minkowski p=1
k=1
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.38


In [21]:
# Minkowski p=2
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.46


In [23]:
# Minkowski p=3
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.56


In [29]:
# Minkowski p=4
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.47


In [31]:
# Minkowski p=5
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.53


In [25]:
# City block (manhattan)
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.38


In [27]:
# Chebyshev
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

21.18


#### L1 family

In [40]:
# Sorensen
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

26.65


In [64]:
# Gower
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.38


In [66]:
# Soergel
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.35


In [74]:
# Kulczynski d
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.35


In [71]:
# Canberra
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.42


In [82]:
# Lorentzian
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

5.91


#### Intersection family

In [84]:
# Intersection
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

39.35


In [90]:
# Wave Hedges
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.95


In [93]:
# Czekanowski s
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

7.46


In [121]:
# Czekanowski d
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

56.34


In [95]:
# Motyka s
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

7.46


In [123]:
# Motyka d
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

56.34


In [97]:
# Kulczynski s
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

<ipython-input-96-cdb19d2d6192>:60: RuntimeWarning: divide by zero encountered in long_scalars
  dist = sum(list(map(truediv, min_el, abs_subs)))
<ipython-input-96-cdb19d2d6192>:60: RuntimeWarning: invalid value encountered in double_scalars
  dist = sum(list(map(truediv, min_el, abs_subs)))


35.61


In [103]:
# Ruzicka
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.35


In [105]:
# Tanimoto
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.35


#### Inner product family

In [111]:
# Inner product
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

48.84


In [109]:
# Harmonic mean
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

55.84


In [113]:
# Cosine
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.46


In [119]:
# Kumar-Hassebrook
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

49.36


In [117]:
# Jaccard s
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

49.36


In [125]:
# Jaccard d
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.46


In [130]:
# Dice s
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

49.36


In [132]:
# Dice d
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))

6.46


#### Fidelity family or Squared-chord family

In [ ]:
# Bhattacharyya
err = positioning_error(k, df_tstrss_DSI1, df_trnrss_DSI1, df_tstcrd_DSI1, df_trncrd_DSI1)
print(round(st.fmean(err), 2))